In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import random
import datetime
%load_ext tensorboard

In [2]:
from google.cloud import datastore
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = \
    "/home/alice/Downloads/HCL-customer-application-c05a56fa9ccc.json"


client = datastore.Client()
query = client.query(kind="pricedata")
x = []
y = []
for i in query.fetch():
    x.append([i['smolprice'], i['chonkprice'], i['absprice'],i["balance"]])
    y.append([i['coins']])


In [3]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(4, activation='relu',input_shape=(4,)))
model.add(layers.Dense(3, activation='relu',input_shape=(4,)))

# Add an output layer with 10 output units:
model.add(layers.Dense(1))

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

In [5]:
import numpy as np

In [6]:
data = np.asarray(x)
target = np.asarray(y)
print(data)
print(target)
print(data.shape)
print(target.shape)

[[   5  205  155 1549]
 [   5  355  355 1549]
 [ 205   55   55   56]
 [   5  355  355 1549]
 [   5  355  355 1549]
 [  50  300  500   38]
 [   5  505  555 1204]]
[[ 40]
 [150]
 [230]
 [280]
 [  0]
 [904]
 [360]]
(7, 4)
(7, 1)


In [7]:

def Normalize(data, mean_data =None, std_data =None):
    if not mean_data:
        mean_data = np.mean(data)
    if not std_data:
        std_data = np.std(data)
    if std_data == 0:
        norm_data = (data-mean_data)
    else:
        norm_data = (data-mean_data)/std_data
    return norm_data, mean_data, std_data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

X_train, xmean_data, xstd_data = Normalize(X_train)
X_test, _, _ = Normalize(X_test, xmean_data, xstd_data)

Y_train, ymean_data, ystd_data = Normalize(y_train)
Y_test,_,_ = Normalize(y_test, ymean_data, ystd_data)

In [8]:
print(X_train)
print(Y_train)

[[-0.41859077 -0.73602563 -0.73602563 -0.7339094 ]
 [-0.84183725 -0.10115591 -0.10115591  2.42562558]
 [-0.84183725 -0.10115591 -0.10115591  2.42562558]
 [-0.74660679 -0.21754869  0.20569779 -0.77200158]
 [-0.84183725  0.21627895  0.32209057  1.6955254 ]]
[[-0.31832276]
 [-0.57607398]
 [-1.05935752]
 [ 1.85323128]
 [ 0.10052298]]


In [9]:
print(X_test)
print(Y_test)

[[-0.84183725 -0.10115591 -0.10115591  2.42562558]
 [-0.84183725 -0.41859077 -0.52440239  2.42562558]]
[[-0.15722824]
 [-0.93048191]]


In [10]:
model.fit(X_train, Y_train,
          validation_data=(X_test,Y_test), 
          batch_size=20, 
          epochs=25,
          verbose=1)

Epoch 1/25
1/1 [==============================] - 0s 219ms/step - loss: 2.2670 - mae: 1.3411 - val_loss: 2.4195 - val_mae: 1.5124
Epoch 2/25
1/1 [==============================] - 0s 32ms/step - loss: 2.1226 - mae: 1.2920 - val_loss: 2.1759 - val_mae: 1.4297
Epoch 3/25
1/1 [==============================] - 0s 34ms/step - loss: 1.9926 - mae: 1.2451 - val_loss: 1.9569 - val_mae: 1.3510
Epoch 4/25
1/1 [==============================] - 0s 29ms/step - loss: 1.8759 - mae: 1.2004 - val_loss: 1.7610 - val_mae: 1.2764
Epoch 5/25
1/1 [==============================] - 0s 29ms/step - loss: 1.7718 - mae: 1.1581 - val_loss: 1.5861 - val_mae: 1.2059
Epoch 6/25
1/1 [==============================] - 0s 29ms/step - loss: 1.6792 - mae: 1.1180 - val_loss: 1.4306 - val_mae: 1.1396
Epoch 7/25
1/1 [==============================] - 0s 39ms/step - loss: 1.5971 - mae: 1.0802 - val_loss: 1.2928 - val_mae: 1.0774
Epoch 8/25
1/1 [==============================] - 0s 34ms/step - loss: 1.5246 - mae: 1.0447 - va

In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 20        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 15        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 4         
Total params: 39
Trainable params: 39
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
%tensorboard --logdir logs/fit


In [13]:
key = client.key("thoughtcoins", 5713227891605504)
entity = client.get(key=key)
balance = entity["balance"]

In [14]:
balance

659

In [15]:
data = []
maxval = 0
stepsize = 50
maxvaldat = [[5, 5, 5]]
for smol in range(5, 600, stepsize):
    for chonk in range(5, 600, stepsize):
        for abso in range(5, 600, stepsize):
            x = np.array([[smol, chonk, abso,balance]])
            dat = Normalize(x, xmean_data, xstd_data)[0]
            pred = model.predict(dat)
            data.append([smol,chonk,abso,pred[0][0] *ystd_data + ymean_data])
            if(pred > maxval):
                maxval = pred
                maxvaldat = [[smol, chonk, abso]]


In [16]:
def keyfunc(e):
    return e[3]

In [17]:
data.sort(reverse=True,key=keyfunc)

In [18]:
data

[[5, 5, 555, 293.4400374723081],
 [5, 55, 555, 292.3807494369875],
 [5, 5, 505, 291.68644591110257],
 [55, 5, 555, 291.3537391006297],
 [5, 555, 555, 291.33716319103195],
 [5, 105, 555, 291.32146140166685],
 [5, 55, 505, 290.53788430016505],
 [5, 505, 555, 290.4953391571991],
 [55, 55, 555, 290.2944487528217],
 [5, 155, 555, 290.2621710538589],
 [5, 5, 455, 290.0092543082244],
 [5, 455, 555, 289.6535104983915],
 [55, 5, 505, 289.57831765848067],
 [5, 105, 505, 289.3893226892275],
 [5, 555, 505, 289.35710511493676],
 [105, 5, 555, 289.26743610397654],
 [55, 105, 555, 289.2351584050137],
 [5, 205, 555, 289.20288070605085],
 [55, 555, 555, 289.18531967682577],
 [5, 55, 455, 288.8606973222617],
 [5, 405, 555, 288.81169108953344],
 [5, 505, 505, 288.5152764561292],
 [55, 55, 505, 288.42975604754315],
 [55, 505, 555, 288.34349564299293],
 [5, 5, 405, 288.3320719552958],
 [5, 155, 505, 288.27457426887855],
 [105, 55, 555, 288.2081503811433],
 [55, 155, 555, 288.17586805720566],
 [5, 255, 555,

In [20]:
newprice = random.choice(data[:20])
newprice

[55, 105, 555, 289.2351584050137]

In [38]:
client = datastore.Client()
key = client.key('prices', 5631671361601536)
entity = datastore.Entity(key=key)
entity.update({
    "smolprice": newprice[0],
    "chonkprice": newprice[1],
   "absprice": newprice[2]    })
client.put(entity)

## import matplotlib.pyplot as plt

In [111]:
(8-2)/3

2.0

In [112]:
2.0*3+2

8.0